# Part of Speech Tagging with Hidden Markov Models 
---
### Aim

To determine the syntactic category of a word from the words in its surrounding context. In this project, I used the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. 

### Reason for using HMMs
They are faster, not memory intensive and provides satisfactory accuracy for our use case.

Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more.  

### Alternatives Considered
We can use a RNN but the inference and training time will be longer. Moreover, RNN might improve the model accuracy by atmost 2%. Trade-off between improvement and development time tipped scales in HMM favor.

### Use Cases for POS tagging
It is often used to help disambiguate natural language phrases (as it can be done quickly with high accuracy), determine correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

### Python Environment
You can recreate the necessary environment using requirements.txt file.

In [1]:
# Jupyter "magic methods"
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from IPython.core.display import HTML
from itertools import chain
from collections import namedtuple
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

/home/sujit/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Preprocessing
--- 
The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). Another data source [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html)

The `Dataset` class in helpers.py will read and parse the corpus. Can use this class on any text file with following format: words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Example from the Brown corpus. 
```
b100-38532
Perhaps	ADV
it	PRON
was	VERB
right	ADJ
;	.
;	.

b100-35577
...
```

In [3]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


### The Dataset Interface

You can access (mostly) immutable references to the dataset through a simple interface provided through the `Dataset` class, which represents an iterable collection of sentences along with easy access to partitions of the data for training & testing. 

```
Dataset-only Attributes:
    training_set - reference to a Subset object containing the samples for training
    testing_set - reference to a Subset object containing the samples for testing

Dataset & Subset Attributes:
    sentences - a dictionary with an entry {sentence_key: Sentence()} for each sentence in the corpus
    keys - an immutable ordered (not sorted) collection of the sentence_keys for the corpus
    vocab - an immutable collection of the unique words in the corpus
    tagset - an immutable collection of the unique tags in the corpus
    X - returns an array of words grouped by sentences ((w11, w12, w13, ...), (w21, w22, w23, ...), ...)
    Y - returns an array of tags grouped by sentences ((t11, t12, t13, ...), (t21, t22, t23, ...), ...)
    N - returns the number of distinct samples (individual words or tags) in the dataset

Methods:
    stream() - returns an flat iterable over all (word, tag) pairs across all sentences in the corpus
    __iter__() - returns an iterable over the data as (sentence_key, Sentence()) pairs
    __len__() - returns the nubmer of sentences in the dataset
```

For example, consider a Subset, `subset`, of the sentences `{"s0": Sentence(("See", "Spot", "run"), ("VERB", "NOUN", "VERB")), "s1": Sentence(("Spot", "ran"), ("NOUN", "VERB"))}`. The subset will have these attributes:

```
subset.keys == {"s1", "s0"}  # unordered
subset.vocab == {"See", "run", "ran", "Spot"}  # unordered
subset.tagset == {"VERB", "NOUN"}  # unordered
subset.X == (("Spot", "ran"), ("See", "Spot", "run"))  # order matches .keys
subset.Y == (("NOUN", "VERB"), ("VERB", "NOUN", "VERB"))  # order matches .keys
subset.N == 7  # there are a total of seven observations over all sentences
len(subset) == 2  # because there are two sentences
```

<div class="alert alert-block alert-info">
**Note:** The `Dataset` class is _convenient_, but it is **not** efficient. It is not suitable for huge datasets because it stores multiple redundant copies of the same data.
</div>

#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [43]:
data.training_set.sentences['b100-35433']

Sentence(words=('Whenever', 'artists', ',', 'indeed', ',', 'turned', 'to', 'actual', 'representations', 'or', 'molded', 'three-dimensional', 'figures', ',', 'which', 'were', 'rare', 'down', 'to', '800', 'B.C.', ',', 'they', 'tended', 'to', 'reflect', 'reality', '(', 'see', 'Plate', '6a', ',', '9b', ')', ';', ';'), tags=('ADV', 'NOUN', '.', 'ADV', '.', 'VERB', 'ADP', 'ADJ', 'NOUN', 'CONJ', 'VERB', 'ADJ', 'NOUN', '.', 'DET', 'VERB', 'ADJ', 'PRT', 'ADP', 'NUM', 'NOUN', '.', 'PRON', 'VERB', 'PRT', 'VERB', 'NOUN', '.', 'VERB', 'NOUN', 'NUM', '.', 'NUM', '.', '.', '.'))

<div class="alert alert-block alert-info">
**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.
</div>

#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [6]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [7]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [8]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.training_set.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Whenever', 'ADV')
	 ('artists', 'NOUN')
	 (',', '.')
	 ('indeed', 'ADV')
	 (',', '.')
	 ('turned', 'VERB')
	 ('to', 'ADP')


## A Simple Model: Most Frequent Class tagger
---

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

#### Pair Counts

Compute the joint frequency counts for two input sequences.

In [9]:
for i, pair in enumerate(data.training_set.stream()):
    print("\t", pair)
    if i > 5: break

	 ('Whenever', 'ADV')
	 ('artists', 'NOUN')
	 (',', '.')
	 ('indeed', 'ADV')
	 (',', '.')
	 ('turned', 'VERB')
	 ('to', 'ADP')


In [11]:
# no need to send sequences into function. This logic will automatically count occurences of (pos, word) pairs in train dataset
# this method is most efficient and fast than looping over sentences in training data
def pair_counts(): #sequences_A, sequences_B
    """Returns a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    this function returns a dictionary such that pair_counts[NOUN][time] == 1244
    """
    # creating a dataframe from list of tuples. Each tuple is a word, pos pair
    tmp_df = pd.DataFrame([pair for pair in data.training_set.stream()], columns=['word', 'pos'])
    # creating a dummy column
    tmp_df['count'] = 1
    # using groupby to count (pos, word) occurences
    df = tmp_df.groupby(by=['pos','word'],as_index=False)['count'].count()
    # initializing a dict of dicts
    tmp = defaultdict(dict)
    for i in range(len(df)):
        tmp[df.loc[i,'pos']][df.loc[i,'word']] = df.loc[i,'count'] 
    return tmp

In [12]:
# Calculate C(t_i, w_i)
emission_counts = pair_counts()

assert len(emission_counts) == 12, \
       "There should be 12 tags in your dictionary."

HTML('<div class="alert alert-block alert-success">Emission counts look good!</div>')

#### IMPLEMENTATION: Most Frequent Class Tagger

Find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.


In [13]:
def _word_pos_():
    """
    for training set this function returns a dict with word as key and 
    most frequent pos associated with as value.
    """
    # creating a dataframe from list of tuples. Each tuple is a word, pos pair
    tmp_df = pd.DataFrame([pair for pair in data.training_set.stream()], columns=['word', 'pos'])
    # creating a dummy column
    tmp_df['count'] = 1
    # using groupby to count (pos, word) occurences
    df = tmp_df.groupby(by=['pos','word'],as_index=False)['count'].count()

    # get max pos associated with a word
    df = df.loc[df.groupby(['word'], as_index=False)['count'].idxmax().tolist()]
    df.reset_index(drop=True, inplace=True)

    tmp = {}
    for i in range(len(df)):
        tmp[df.loc[i,'word']] = df.loc[i,'pos'] 
    return tmp

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [44]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label 
# most frequently assigned to that word

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))

In [45]:
# calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

word_counts = pair_counts() 

# used my custom function to get most frequent tag (pos) associated with a word
mfc_table = _word_pos_() 


mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

assert len(mfc_table) == len(data.training_set.vocab), ""
assert all(k in data.training_set.vocab for k in mfc_table.keys()), ""
assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, ""
HTML('<div class="alert alert-block alert-success">MFC tagger has all the correct words!</div>')

#### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function.

In [15]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

#### Example Decoding Sequences with MFC Tagger

In [16]:
for key in data.testing_set.keys[:1]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')




#### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [17]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger


In [18]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">MFC tagger accuracy looks correct!</div>')

training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.02%


## HMM tagger
---
The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where functions are implemented to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t_i^n = \underset{t_i^n}{\mathrm{argmax}} \prod_{i=1}^n P(w_i|t_i) P(t_i|t_{i-1})$$

Reference: Speech & Language Processing [Chapter 10](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

### Unigram Counts

Function below estimates the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [19]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    return dict(Counter(list(np.hstack(sequences))))

In [20]:
# call unigram_counts with a list of tag sequences from the training set
tag_unigrams = unigram_counts([x for x in data.training_set.Y]) # TODO: YOUR CODE HERE

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">tag unigrams look good!</div>')

### Bigram Counts

The function below estimates the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [21]:
def bigram_counts(sequences, ret_dict=True):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    if ret_dict:
        return dict(Counter([(x,y) for i in sequences for x,y in zip(i[:-1], i[1:])]))
    else:
        tmp = [(x,y) for i in sequences for x,y in zip(i[:-1], i[1:])]
        dftr = pd.DataFrame(tmp, columns=['start', 'end'])
        dftr = dftr.assign(total=1)
        return dftr.groupby(by=['start','end'],as_index=False)['total'].count()

In [22]:
# call bigram_counts with a list of tag sequences from the training set
tag_bigrams = bigram_counts([x for x in data.training_set.Y]) # TODO: YOUR CODE HERE

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">tag bigrams look good!</div>')

### Sequence Starting Counts
Function below to estimates the bigram probabilities of a sequence starting with each tag.

In [23]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    return dict(Counter([i[0] for i in sequences]))

In [24]:
#Calculate the count of each tag starting a sequence
tag_starts = starting_counts([x for x in data.training_set.Y]) # TODO: YOUR CODE HERE

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">starting tag counts look good!</div>')

### Sequence Ending Counts
The function below to estimate the bigram probabilities of a sequence ending with each tag.

In [25]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    return dict(Counter([i[-1] for i in sequences]))

In [26]:
# Calculate the count of each tag ending a sequence
tag_ends = ending_counts([x for x in data.training_set.Y]) # TODO: YOUR CODE HERE

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">ending tag counts look good!</div>')

### Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [27]:
# reads in the data from data subset and creates a dataframe with word, pos and count(how many times they occured)
def _data2df_():
    # creating a dataframe from list of tuples. Each tuple is a word, pos pair
    tmp_df = pd.DataFrame([pair for pair in data.training_set.stream()], columns=['word', 'pos'])
    # creating a dummy column
    tmp_df['count'] = 1
    # using groupby to count (pos, word) occurences
    return tmp_df.groupby(by=['pos','word'],as_index=False)['count'].count() 

def _emission_p_(_df_, pos):
    _df_ = _df_.loc[_df_.pos == pos]
    _df_ = _df_.assign(prob=_df_['count']/_df_['count'].sum())
    _df_.reset_index(drop=True, inplace=True)
    tmp={}
    assert len(_df_) == len(_df_.word.unique()), "Uh oh. Length and unique words are not equal, it should be."
    for i in range(len(_df_)):
        tmp[_df_.loc[i, 'word']] = _df_.loc[i, 'prob']
    return tmp

In [31]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

In [32]:
# read training data to a df
df_t = _data2df_()

In [33]:
# create states with emission probability distributions P(word | tag) and add to the model
# add hidden states (pos or tags)
for i in data.training_set.tagset:
    if i == '.':
        # emission probability distributions, P(word | pos)
        tmp_emissions = DiscreteDistribution(_emission_p_(df_t, i))
        dot_state =  State(tmp_emissions, name=i)
        
        # add the state to the model
        basic_model.add_state(dot_state)
    else:
        # emission probability distributions, P(word | pos)
        tmp_emissions = DiscreteDistribution(_emission_p_(df_t, i))
        vars()[i] = State(tmp_emissions, name=i)

        # add the state to the model
        basic_model.add_state(vars()[i])

In [34]:
# add edges between states for the observed transition frequencies P(tag_i | tag_i-1)
len_train = sum(tag_starts.values())
# adding transitional prob for initial state (model.start) to all pos, P(start to pos) = P(pos | start)
for i in tag_starts:
    if i =='.':
        # using tag_starts we created earlier
        basic_model.add_transition(basic_model.start, dot_state, tag_starts[i]/len_train) 
    else:
        # using tag_starts we created earlier
        basic_model.add_transition(basic_model.start, vars()[i], tag_starts[i]/len_train) 

In [37]:
# adding transitional prob from penultimate state to end state (model.end) to all pos, 
# P(penul_pos to end) = P(end | penul_pos)
for i in tag_starts:
    if i =='.':
        # using tag_ends we created earlier
        basic_model.add_transition(dot_state, basic_model.end, tag_ends[i]/len_train) 
    else:
        # using tag_ends we created earlier
        basic_model.add_transition(vars()[i], basic_model.end, tag_ends[i]/len_train) 

In [38]:
# adding transition probabilities
dftr = bigram_counts([x for x in data.training_set.Y], ret_dict=False) # returns a df
for i in tag_starts:
    _dftr_ = dftr.loc[dftr.start == i]
    _dftr_.reset_index(drop=True, inplace=True)
    _dftr_ = _dftr_.assign(prob=_dftr_.total/_dftr_.total.sum())
    for j in tag_starts:
        if j =='.':
            if i == '.':
                basic_model.add_transition(dot_state, dot_state, _dftr_.loc[(_dftr_.start==i) & (_dftr_.end==j)]['prob'].tolist()[0])
            else:
                basic_model.add_transition(vars()[i], dot_state, _dftr_.loc[(_dftr_.start==i) & (_dftr_.end==j)]['prob'].tolist()[0])
        else:
            if i == '.':
                basic_model.add_transition(dot_state, vars()[j], _dftr_.loc[(_dftr_.start==i) & (_dftr_.end==j)]['prob'].tolist()[0])
            else:
                basic_model.add_transition(vars()[i], vars()[j], _dftr_.loc[(_dftr_.start==i) & (_dftr_.end==j)]['prob'].tolist()[0])

In [39]:
# finalize the model
basic_model.bake()

assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">HMM network topology looks good!</div>')

In [40]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_testing_acc > 0.955, "Uh oh. Your HMM accuracy on the testing set doesn't look right."

training accuracy basic hmm model: 97.54%
testing accuracy basic hmm model: 95.95%


### Example Decoding Sequences with the HMM Tagger

In [41]:
for key in data.testing_set.keys[:1]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')




## Future Improvements and Other referneces
---
There are additional enhancements that can be incorporated into the tagger that can improve performance on larger tagsets where the data sparsity problem is more significant. The data sparsity problem arises because the same amount of data split over more tags means there will be fewer samples in each tag, and there will be more missing data  tags that have zero occurrences in the data.

- [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (pseudocounts)
    Laplace smoothing is a technique where you add a small, non-zero value to all observed counts to offset for unobserved values.

- Backoff Smoothing
    Another smoothing technique is to interpolate between n-grams for missing data. This method is more effective than Laplace smoothing at combatting the data sparsity problem. Refer to chapters 4, 9, and 10 of the [Speech & Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book for more information.

- Extending to Trigrams
    HMM taggers have achieved better than 96% accuracy on this dataset with the full Penn treebank tagset using an architecture described in [this](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf) paper. Altering your HMM to achieve the same performance would require implementing deleted interpolation (described in the paper), incorporating trigram probabilities in your frequency tables, and re-implementing the Viterbi algorithm to consider three consecutive states instead of two.

Refer to [Chapter 5](http://www.nltk.org/book/ch05.html) of the NLTK book for more information on the available tagsets.

In [ ]:
# Get Brown Corpus with a Larger Tagset
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]